In [91]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt

In [92]:
def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
def count(D, a, b):
    counter = 0
    n = len(D)
    for i in range(n):
        if D[i] >=a and D[i]<=b:
            counter+=1
    return counter
def unboundedDomainRadius(eps, beta, D, B):
    TNoise = 2/eps*LapNoise()
    T = len(D)- 6/eps*np.log(2/beta)+TNoise
#     print(T)
    for i in range(int(math.log2(B))):
        Qi = count(D,0, math.pow(2 ,i))+4/eps*LapNoise()
#         print(Qi)
        if Qi>T:
            break
    return  math.pow(2,i-1)   

    
def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def universal(I, eps, B, beta):
    rad = unboundedDomainRadius(eps/2, beta/2, I, B)
#     print(rad)
    clipped = clip(I,0,rad)
    
    return sum(clipped)+ 2*rad/(eps)*LapNoise()

In [93]:
num_user=[10000, 100000, 1000000, 10000000, 100000000]
ratio = 0.1
num_repeats = 1
B = 10000000
B_vary = [100000, 1000000, 10000000, 100000000, 1000000000]
ratio = 0.1
fc = 0.54
fm = 0.37
fl = 0.09


#The current setting is for experiments on Bank dataset.
#To try simulated datasets, uncomment this block and comment the following block in each subroutine.

# n = int(num_user*ratio)

# #eps of non-zero users
# eps = np.random.normal(1,0.3,int(n))
# eps[eps<1/(10*math.sqrt(n))]=1/(10*math.sqrt(n))
# # t = sum(eps)/n
# eps_sort = np.sort(eps)
# print(eps_sort)
# #This time also need to generate data
# I = np.random.normal(1000, 200, n)
# I[I<0]=1
# I = np.int_(I)
# real_result = sum(I)
# print(sum(I))

# #to enable EM, we also need eps for zero users
# eps_zero = np.random.normal(1,0.3,num_user-int(n))
# eps_zero[eps_zero<1/(10*math.sqrt(n))]=1/(10*math.sqrt(n))
# eps_zero_sort = np.sort(eps_zero)

# t = (sum(eps)+sum(eps_zero))/num_user

In [94]:
# start = time.time()
# input_path = "./bank_marketing.csv"
# input_file = open(input_path,'r')
# n = len(input_file.readlines())-1
# input_file.close()
# I = np.zeros(n)  
# i = 0
# input_file = open(input_path,'r')
# for line in input_file.readlines():
#         elements = line.split(",")
#         if i>0:
#             I[i-1] = int(elements[5])
#         i+=1

# # I.sort()

# print(sum(I))
# end = time.time()
# print(end-start)

# print(len(I))
# #positive and negative
# Ip = I[I>=0]
# In = -I[I<0]
# npo = len(Ip)
# nn = len(In)
# print(len(Ip))
# print(len(In))


In [95]:
def sample(eps, I, n, t, B, beta=0.1):
    temp = copy.deepcopy(I)
    for i in range(n):
        if eps[i]<t:
            temp[i] = np.random.binomial(1, (math.exp(eps[i])-1)/(math.exp(t)-1))*I[i]
            
    return universal(temp, t, B, beta)

In [78]:
times = []
errors = []
relative_error = []
real_results = []
#should be changed to range(5) in simluated datasets
for i in range(5):
    #uncomment this line in simluated datasets
#     n = int(num_user[i]*ratio)
    
    #Gaussian eps
    #eps of non-zero users
    eps = np.random.normal(1,0.3,int(n))
    eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
    epsp = eps[I>=0]
    epsn = eps[I<0]
    # t = sum(eps)/n
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
    eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
    eps_zero_sort = np.sort(eps_zero)

    
#     #Uniform eps
#     nc = int(n*fc)
#     nm = int(n*fm)
#     nl = int(n*fl)
#     epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
#     epsm = np.random.uniform(0.5, 2, nm)
#     epsl = np.ones(nl)*2
#     eps = np.append(np.append(epsc, epsm), epsl)
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     nc_z = int((num_user[i]-n)*fc)
#     nm_z = int((num_user[i]-n)*fm)
#     nl_z = int((num_user[i]-n)*fl)
#     epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
#     epsm_z = np.random.uniform(0.5, 2, nm_z)
#     epsl_z = np.ones(nl_z)*2
#     eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
#     eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
#     I = np.random.normal(1000, 200, n)
#     I = np.int_(I)
#     I[I<=0]=1
    real  = sum(I)
    print(real)
    real_results.append(real)
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = sample(eps, I, n, t, B)
#         resultp = sample(epsp, Ip, npo, t, B, beta = 0.05)
#         resultn = sample(epsn, In, nn, t, B, beta = 0.05)
#         result = resultp-resultn
        error += abs(result-real)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    


61589682.0
times
[0.12042498588562012]
errors
[12070308.716286168]
relative_error
[0.19597939661851424]


In [79]:
def pdp_sum(eps, I, n, eps_min, eps_max, beta, B):
    t = math.ceil(math.log(eps_max*B/eps_min, 2))
    result=0
    for i in range(t+1):
        tau = np.zeros(n)
        temp = copy.deepcopy(I)
        for j in range(n):
            tau[j] = math.floor(math.pow(2, i)*eps[j]/eps_max)
            temp[j] = min(tau[j], temp[j])
#         print("i = "+str(i))
        sum_tilde = sum(temp)+ t*math.pow(2, i)/eps_max*LapNoise()-t*math.pow(2, i)/eps_max*np.log(t/beta)
#         print(sum_tilde)
        result = max(result,sum_tilde)
    return result

In [87]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(2,3):
#     n = int(num_user[i]*ratio)

    #Gaussian eps
    #eps of non-zero users
    eps = np.random.normal(1,0.3,int(n))
    eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
    epsp = eps[I>=0]
    epsn = eps[I<0]
    # t = sum(eps)/n
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
    eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
    eps_zero_sort = np.sort(eps_zero)
    
#     #Uniform eps
#     nc = int(n*fc)
#     nm = int(n*fm)
#     nl = int(n*fl)
#     epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
#     epsm = np.random.uniform(0.5, 2, nm)
#     epsl = np.ones(nl)*2
#     eps = np.append(np.append(epsc, epsm), epsl)
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     nc_z = int((num_user[i]-n)*fc)
#     nm_z = int((num_user[i]-n)*fm)
#     nl_z = int((num_user[i]-n)*fl)
#     epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
#     epsm_z = np.random.uniform(0.5, 2, nm_z)
#     epsl_z = np.ones(nl_z)*2
#     eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
#     eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
#     I = np.random.normal(1000, 200, n)
#     I = np.int_(I)
#     I[I<=0]=1
    real  = sum(I)
    real_results.append(real)
    
    start = time.time()
    error = 0
#     print(real)
    for j in range(num_repeats):
#         result = pdp_sum(eps, I, n, eps_min, eps_max, 0.1, B)
        resultp = pdp_sum(epsp, Ip, npo, eps_min, eps_max, 0.05, B)
        resultn = pdp_sum(epsn, In, nn, eps_min, eps_max, 0.05, B)
        result = resultp-resultn
        error += abs(result-real)
#         print(abs(result-real))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    


sum p 62786056.0result p 56361358.712131456relative 0.10232681740462475
times
[0.8957712650299072]
errors
[6115888.6451829]
relative_error
[0.09930053941799699]


In [88]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(2,3):
#     n = int(num_user[i]*ratio)

    #Gaussian eps
    #eps of non-zero users
    eps = np.random.normal(1,0.3,int(n))
    eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
    epsp = eps[I>=0]
    epsn = eps[I<0]
    # t = sum(eps)/n
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
    eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
    eps_zero_sort = np.sort(eps_zero)
    
#     #Uniform eps
#     nc = int(n*fc)
#     nm = int(n*fm)
#     nl = int(n*fl)
#     epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
#     epsm = np.random.uniform(0.5, 2, nm)
#     epsl = np.ones(nl)*2
#     eps = np.append(np.append(epsc, epsm), epsl)
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     nc_z = int((num_user[i]-n)*fc)
#     nm_z = int((num_user[i]-n)*fm)
#     nl_z = int((num_user[i]-n)*fl)
#     epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
#     epsm_z = np.random.uniform(0.5, 2, nm_z)
#     epsl_z = np.ones(nl_z)*2
#     eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
#     eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
#     I = np.random.normal(1000, 200, n)
#     I = np.int_(I)
#     I[I<=0]=1
    real  = sum(I)
    real_results.append(real)
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
#         result = universal(I, eps_min, B, 0.1)
        resultp = universal(Ip, eps_min, B, 0.05)
        resultn = universal(In, eps_min, B, 0.05)
        result = resultp-resultn
        error += abs(result-real)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    

times
[0.008941650390625]
errors
[61575698.58201955]
relative_error
[0.9997729584318937]


In [10]:
#blocks below are not used in final experiments

In [11]:
# def subroutine(I, eps, B, beta):
#     t = math.ceil(math.log(B, 2))
#     result = 0
#     for i in range(t):
#         tau = math.pow(2,i)
#         temp = copy.deepcopy(I)
#         temp[temp>tau]=tau
# #         print(tau)
# #         print(sum(temp))
#         tilde_sum = sum(temp) + math.log(B, 2)*tau/eps*LapNoise() -math.log(B, 2)*tau/eps*np.log(math.log(B, 2)/beta)
#         result = max(result,tilde_sum)
#     return result

# #     t = math.ceil(math.log(B, 2))
# #     result = 0
# #     #svt
# #     for i in range(t):
# #         tau = math.pow(2,i)
# #         temp = copy.deepcopy(I)
# #         temp[temp>tau]=tau
# # #         print(tau)
# # #         print(sum(temp))
# #         tilde_sum = sum(temp) + math.log(B, 2)*tau/eps*LapNoise() -math.log(B, 2)*tau/eps*np.log(math.log(B, 2)/beta)
# #         result = max(result,tilde_sum)
# #     return result

In [12]:
# def direct_sum(eps, I, n, eps_min, eps_max, beta, B):
#     t = math.ceil(math.log(eps_max/eps_min, 2))
#     result=0
#     for i in range(t+1):
#         k = math.pow(2,i)*eps_min
#         temp = copy.deepcopy(I)
#         for j in range(n):
#             if eps[j]<k:
#                 temp[j] = 0
# #         print("k = "+str(k))
# #         print("current sum = "+str(sum(temp)))
#         sum_tilde = universal(temp, k/t, B, beta/t)
# #         print(sum_tilde)
#         result = max(result,sum_tilde)
        
#     return result


In [13]:
# times = []
# errors = []
# relative_error = []
# real_results = []
# for i in range(2,3):
# #     n = int(num_user[i]*ratio)

# #     #Gaussian eps
# #     #eps of non-zero users
# #     eps = np.random.normal(1,0.3,int(n))
# #     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
# #     # t = sum(eps)/n
# #     eps_sort = np.sort(eps)
# #     #to enable EM, we also need eps for zero users
# #     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
# #     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
# #     eps_zero_sort = np.sort(eps_zero)
    
#     #Uniform eps
#     nc = int(n*fc)
#     nm = int(n*fm)
#     nl = int(n*fl)
#     epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
#     epsm = np.random.uniform(0.5, 2, nm)
#     epsl = np.ones(nl)*2
#     eps = np.append(np.append(epsc, epsm), epsl)
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     nc_z = int((num_user[i]-n)*fc)
#     nm_z = int((num_user[i]-n)*fm)
#     nl_z = int((num_user[i]-n)*fl)
#     epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
#     epsm_z = np.random.uniform(0.5, 2, nm_z)
#     epsl_z = np.ones(nl_z)*2
#     eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
#     I = np.random.normal(1000, 200, n)
#     I = np.int_(I)
#     I[I<=0]=1
#     real  = sum(I)
#     real_results.append(real)
    
#     start = time.time()
#     error = 0
#     #print(real)
#     for j in range(num_repeats):
#         result = direct_sum(eps, I, n, eps_min, eps_max, 0.1, B)
#         error += abs(result-real)
#         #print(abs(result-real))
    
    
#     end= time.time()
#     times.append((end-start)/num_repeats)
#     errors.append(error/num_repeats)
#     relative_error.append((error/num_repeats)/real)
# #     print("Average Time for i = "+str(i))
# #     print((end-start)/num_repeats)
# #     print("Average Error for i = "+str(i))
# #     print(error/num_repeats)

    
# print("times")
# print(times)
# print("errors")
# print(errors)
# print("relative_error")
# print(relative_error)
    


In [14]:
# #seems it can not do sum in polynomial time
# def pdp_em_sum(I):
#     #first define the utility scores
#     score = np.zeros(num_user+1)
#     temp1 = 0
#     temp2 = 0
#     for i in range(num_user+1):
#         if i<n:
#             temp1+= eps_sort[i]
#             score[n-1-i] = -temp1
            
#         elif i>n:
#             temp2+= eps_zero_sort[i-int(n)-1]
#             score[i] = -temp2
# #     print(score[n])
# #     print(score[0])
# #     print(score[num_user])
#     weights = np.zeros(num_user+1)
#     total_weight = 0
#     for i in range(num_user+1):
#         total_weight +=math.exp(1/2*score[i])
#         weights[i] = math.exp(1/2*score[i])
#     weights = weights/total_weight
# #     print(weights[n])
#     idx = np.random.choice(list(range(num_user+1)), p=weights)
#     if idx== n:
#         return sum(I)
#     elif idx > n:
#         return np.random.randint(sum(I)+B*(idx-n), sum(I)+B*(idx-n), dtype=np.int64)